# Logistic curve fit and XGBoost hybrid fit

In previous weeks we found that a logistic curve fit works quite well on a per country level, and that adding a global XGBoost fit with [augmented data](https://www.kaggle.com/nxpnsv/country-health-indicators) is an improvement. The main idea for improvement in this notebook is to make an optimal interpolation between the two methods.

## Set up environment

In [1]:
# Imports
import os
from typing import Dict, List, Tuple
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
from scipy.optimize.minpack import curve_fit
from scipy.optimize import least_squares
from xgboost import XGBRegressor

/var/folders/x9/c9tbhtn54lv8tgynqv_x0bt80000gn/T/ipykernel_23064/1070436655.py:7: DeprecationWarning: Please use `curve_fit` from the `scipy.optimize` namespace, the `scipy.optimize.minpack` namespace is deprecated.
  from scipy.optimize.minpack import curve_fit


In [57]:
def RMSLE(actual: np.ndarray, prediction: np.ndarray) -> float:
    """Calculate RMSLE between actual and predicted values"""
    return np.sqrt(
        np.mean(
            np.power(np.log1p(np.maximum(0, prediction)) - np.log1p(actual), 2)
        )
    )

## Load data

In [58]:
df = pd.read_csv('train.csv')
df.head(10)
df.shape
# csv = pd.read_csv('country_health_indicators_v3.csv')
# csv.head(10)
# csv["DateOfYear"] 
# csv.columns

(23562, 6)

In [43]:
df = pd.read_csv('train.csv')

print(df['Date'])

df["Country_Region"] = np.where(
    df["Province_State"].isnull(),
    df["Country_Region"],
    df["Country_Region"] + "_" + df["Province_State"],
)

# df['country'] = df["Country_Region"]
df.drop(columns="Province_State", inplace=True)

if "ConfirmedCases" in df:
    df["ConfirmedCases"] = df.groupby("Country_Region")[
        "ConfirmedCases"
    ].cummax()

if "Fatalities" in df:
    df["Fatalities"] = df.groupby("Country_Region")["Fatalities"].cummax()

# Add day of year column
if not "DayOfYear" in df:
    df["DayOfYear"] = pd.to_datetime(df["Date"]).dt.dayofyear

# Convert date column to date type and return cleaned dataframe
df["Date"] = pd.to_datetime(df["Date"]).dt.date

0        2020-01-22
1        2020-01-23
2        2020-01-24
3        2020-01-25
4        2020-01-26
            ...    
23557    2020-04-03
23558    2020-04-04
23559    2020-04-05
23560    2020-04-06
23561    2020-04-07
Name: Date, Length: 23562, dtype: object


In [47]:
country_health_indicators = pd.read_csv('country_health_indicators_v3.csv')
country_health_indicators.head(10)
country_health_indicators.shape

(180, 70)

In [46]:
train = pd.merge(df, country_health_indicators, on="Country_Region", how="left")
train.shape

(23562, 75)

In [49]:
train.head(5)

,Id,Country_Region,Date,ConfirmedCases,Fatalities,DayOfYear,first_1ConfirmedCases,first_1Fatalities,first_10ConfirmedCases,first_50ConfirmedCases,...,total fertility rate,obesity - adult prevalence rate,school_shutdown_1case,school_shutdown_10case,school_shutdown_50case,school_shutdown_1death,FF_DayOfYear,case1_DayOfYear,case10_DayOfYear,case50_DayOfYear
0,1,Afghanistan,2020-01-22,0.0,0.0,22,2020-02-24,2020-03-22,2020-03-14,2020-03-24,...,4.82,5.5,19.0,-0.0,-10.0,-8.0,82.0,55.0,74.0,84.0
1,2,Afghanistan,2020-01-23,0.0,0.0,23,2020-02-24,2020-03-22,2020-03-14,2020-03-24,...,4.82,5.5,19.0,-0.0,-10.0,-8.0,82.0,55.0,74.0,84.0
2,3,Afghanistan,2020-01-24,0.0,0.0,24,2020-02-24,2020-03-22,2020-03-14,2020-03-24,...,4.82,5.5,19.0,-0.0,-10.0,-8.0,82.0,55.0,74.0,84.0
3,4,Afghanistan,2020-01-25,0.0,0.0,25,2020-02-24,2020-03-22,2020-03-14,2020-03-24,...,4.82,5.5,19.0,-0.0,-10.0,-8.0,82.0,55.0,74.0,84.0
4,5,Afghanistan,2020-01-26,0.0,0.0,26,2020-02-24,2020-03-22,2020-03-14,2020-03-24,...,4.82,5.5,19.0,-0.0,-10.0,-8.0,82.0,55.0,74.0,84.0


In [ ]:
df = pd.read_csv('train.csv')

print(df['Date'])

df["Country_Region"] = np.where(
    df["Province_State"].isnull(),
    df["Country_Region"],
    df["Country_Region"] + "_" + df["Province_State"],
)

# df['country'] = df["Country_Region"]
df.drop(columns="Province_State", inplace=True)

if "ConfirmedCases" in df:
    df["ConfirmedCases"] = df.groupby("Country_Region")[
        "ConfirmedCases"
    ].cummax()

if "Fatalities" in df:
    df["Fatalities"] = df.groupby("Country_Region")["Fatalities"].cummax()

# Add day of year column
if not "DayOfYear" in df:
    df["DayOfYear"] = pd.to_datetime(df["Date"]).dt.dayofyear

# Convert date column to date type and return cleaned dataframe
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# XGB boost regression

In [50]:
def apply_xgb_model(train, x_columns, y_column, xgb_params):
    X = train[x_columns].to_numpy()
    y = train[y_column].to_numpy()
    xgb_fit = XGBRegressor(**xgb_params).fit(X, y)
    y_hat = xgb_fit.predict(X)
    train[f"yhat_xgb_{y_column}"] = y_hat
    return RMSLE(y, y_hat), xgb_fit

In [51]:
xgb_params = dict(
    gamma=0.2,
    learning_rate=0.15,
    n_estimators=100,
    max_depth=11,
    min_child_weight=1,
    nthread=8,
    objective="reg:squarederror")
x_columns = [
    'DayOfYear', 'cases_growth', 'death_growth',
    'Cardiovascular diseases (%)', 'Cancers (%)',
    'Diabetes, blood, & endocrine diseases (%)', 'Respiratory diseases (%)',
    'Liver disease (%)', 'Diarrhea & common infectious diseases (%)',
    'Musculoskeletal disorders (%)', 'HIV/AIDS and tuberculosis (%)',
    'Malaria & neglected tropical diseases (%)',
    'Nutritional deficiencies (%)', 'pneumonia-death-rates',
    'Share of deaths from smoking (%)', 'alcoholic_beverages',
    'animal_fats', 'animal_products', 'aquatic_products,_other',
    'cereals_-_excluding_beer', 'eggs', 'fish,_seafood',
    'fruits_-_excluding_wine', 'meat', 'milk_-_excluding_butter',
    'miscellaneous', 'offals', 'oilcrops', 'pulses', 'spices',
    'starchy_roots', 'stimulants', 'sugar_&_sweeteners', 'treenuts',
    'vegetable_oils', 'vegetables', 'vegetal_products',
    'hospital_beds_per10k', 'hospital_density', 'nbr_surgeons',
    'nbr_obstetricians', 'nbr_anaesthesiologists', 'medical_doctors_per10k',
    'bcg_coverage', 'bcg_year_delta', 'population',
    'median age', 'population growth rate', 'birth rate', 'death rate',
    'net migration rate', 'maternal mortality rate',
    'infant mortality rate', 'life expectancy at birth',
    'total fertility rate', 'obesity - adult prevalence rate',
    'school_shutdown_1case', 'school_shutdown_10case',
    'school_shutdown_50case', 'school_shutdown_1death', 'FF_DayOfYear',
    'case1_DayOfYear', 'case10_DayOfYear', 'case50_DayOfYear']
xgb_c_rmsle, xgb_c_fit = apply_xgb_model(
    train, x_columns, "ConfirmedCases", xgb_params)
xgb_f_rmsle, xgb_f_fit = apply_xgb_model(
    train, x_columns, "Fatalities", xgb_params)

# Top boosted features

In [52]:
imps=[]
cols = []
for col, fit in (("ConfirmedCases", xgb_c_fit), ("Fatalities", xgb_f_fit)):
    df = pd.DataFrame(list(zip(x_columns, fit.feature_importances_)), columns=[f"feature_{col}", f"importance_{col}"])
    cols.extend(df.columns.to_list())
    imps.append(df.sort_values(by=f"importance_{col}", ascending=False).to_numpy())
importances = pd.DataFrame(np.hstack(imps), columns=cols, index=range(1, len(imps[0])+1))
importances.index.name="rank"
importances.head(20)

,feature_ConfirmedCases,importance_ConfirmedCases,feature_Fatalities,importance_Fatalities
rank,,,,
1,Musculoskeletal disorders (%),0.351357,Cancers (%),0.441975
2,case50_DayOfYear,0.14978,vegetable_oils,0.349166
3,"Diabetes, blood, & endocrine diseases (%)",0.117109,school_shutdown_50case,0.08526
4,medical_doctors_per10k,0.072854,treenuts,0.044628
5,school_shutdown_50case,0.071036,DayOfYear,0.03055
6,Diarrhea & common infectious diseases (%),0.056084,cases_growth,0.009496
7,treenuts,0.052117,Malaria & neglected tropical diseases (%),0.00939
8,nbr_surgeons,0.024811,death_growth,0.009211
9,miscellaneous,0.017751,Musculoskeletal disorders (%),0.00375


Now apply to each `Country_Region`:

# Compare aproaches

In [54]:
print(
    "Confirmed:\n"
    f'XGBoost\t{RMSLE(train["ConfirmedCases"], train["yhat_xgb_ConfirmedCases"])}\n'
    f"Fatalities:\n"
    f'XGBoost\t{RMSLE(train["Fatalities"], train["yhat_xgb_Fatalities"])}\n'
)

Confirmed:
XGBoost	2.8400309516361255
Fatalities:
XGBoost	1.6435190257362453



# Predict test cases

In [15]:
# Merge logistic and hybrid fit into test
test = pd.merge(
    test, 
    train[["Country_Region"] +
          ['ConfirmedCases_p_0', 'ConfirmedCases_p_1', 'ConfirmedCases_p_2']+
          ['Fatalities_p_0','Fatalities_p_1', 'Fatalities_p_2'] + 
          ["Fatalities_alpha"] + 
          ["ConfirmedCases_alpha"]].groupby(['Country_Region']).head(1), on="Country_Region", how="left")

In [56]:
df = pd.read_csv('test.csv')

print(df['Date'])

df["Country_Region"] = np.where(
    df["Province_State"].isnull(),
    df["Country_Region"],
    df["Country_Region"] + "_" + df["Province_State"],
)

# df['country'] = df["Country_Region"]
df.drop(columns="Province_State", inplace=True)

if "ConfirmedCases" in df:
    df["ConfirmedCases"] = df.groupby("Country_Region")[
        "ConfirmedCases"
    ].cummax()

if "Fatalities" in df:
    df["Fatalities"] = df.groupby("Country_Region")["Fatalities"].cummax()

# Add day of year column
if not "DayOfYear" in df:
    df["DayOfYear"] = pd.to_datetime(df["Date"]).dt.dayofyear

# Convert date column to date type and return cleaned dataframe
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# TEST DATA
test = pd.merge(df, country_health_indicators, on="Country_Region", how="left")

0        2020-03-26
1        2020-03-27
2        2020-03-28
3        2020-03-29
4        2020-03-30
            ...    
13153    2020-05-03
13154    2020-05-04
13155    2020-05-05
13156    2020-05-06
13157    2020-05-07
Name: Date, Length: 13158, dtype: object


In [59]:
# Test predictions

test["yhat_xgb_ConfirmedCases"] = xgb_c_fit.predict(test[x_columns].to_numpy())
test["yhat_xgb_Fatalities"] = xgb_f_fit.predict(test[x_columns].to_numpy())


In [63]:
test["yhat_xgb_ConfirmedCases"]

0         96.053040
1        108.567497
2        107.043823
3        127.259033
4        162.017792
            ...    
13153     11.921287
13154     11.921287
13155     11.921287
13156     11.921287
13157     11.921287
Name: yhat_xgb_ConfirmedCases, Length: 13158, dtype: float32